In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal

import rf

import seismic.receiver_fn.rf_util as rf_util
import seismic.receiver_fn.rf_plot_utils as rf_plot_utils
import seismic.receiver_fn.rf_stacking as rf_stacking

## Data loading

In [ ]:
rf_type = rf_type = 'ZRT_it'

In [ ]:
network = 'AQ'
target_station = 'D7'

In [ ]:
src_file = r"..\DATA\AQT_rfs_20151128T042000-20191108T000317_{}_rev1_qual.h5".format(rf_type)

In [ ]:
data_all = rf_util.read_h5_rf(src_file, network, target_station)

In [ ]:
data_all

In [ ]:
type(data_all)

## Select R-channel data

In [ ]:
db = rf_util.rf_to_dict(data_all)

In [ ]:
station_db = db[target_station]

In [ ]:
channel = rf_util.choose_rf_source_channel(rf_type, station_db)
print("Selected channel: {}".format(channel))
channel_data = station_db[channel]
len(channel_data)

In [ ]:
type(channel_data)

In [ ]:
type(channel_data[0])

In [ ]:
channel_data[0].stats.sampling_rate

## Plot initial data

In [ ]:
_ = rf_plot_utils.plot_rf_stack(rf.RFStream(channel_data))

## Look at initial H-k stack

In [ ]:
k_grid, h_grid, hk = rf_stacking.compute_hk_stack(channel_data, include_t3=False, root_order=2)
w = (0.5, 0.5)
w_str = "w={:1.2f},{:1.2f}".format(*w)
stack = rf_stacking.compute_weighted_stack(hk, weighting=w)
hk_fig = rf_plot_utils.plot_hk_stack(k_grid, h_grid, stack, num=len(channel_data),
                                     title="Initial H-k stack, reverb ({})".format(w_str))
plt.show()

## Trial reverb removal on first trace

In [ ]:
x = channel_data[0]

In [ ]:
type(x)

In [ ]:
plt.plot(x[250:750])

In [ ]:
ac = np.correlate(x, x, 'full')/np.dot(x,x)
len(ac)

In [ ]:
ac_causal = ac[len(ac)//2:]
len(ac_causal)

In [ ]:
ac_causal[0]

In [ ]:
plt.plot(ac_causal[:500])

In [ ]:
plt.stem(ac_causal[:50])

In [ ]:
ac_extrema = signal.argrelextrema(ac_causal, np.less)[0]

In [ ]:
ac_extrema

In [ ]:
reverb_removal = np.zeros(ac_extrema[0] + 1)
reverb_removal[0] = 1
reverb_removal[-1] = -ac_causal[ac_extrema[0]]

In [ ]:
plt.stem(reverb_removal)

In [ ]:
len(x)

In [ ]:
x_norev = np.convolve(x, reverb_removal, 'full')

In [ ]:
len(x_norev)

In [ ]:
-len(reverb_removal)+1

In [ ]:
x_norev = x_norev[:(-len(reverb_removal)+1)]
len(x_norev)

In [ ]:
plt.plot(x_norev[250:750])

In [ ]:
fig = plt.figure(figsize=(16,9))
times = x.times() - (x.stats.onset - x.stats.starttime)
mask = (times >= -10.0) & (times <= 25.0)
plt.plot(times[mask], x[mask], alpha=0.8, linewidth=2)
plt.plot(times[mask], x_norev[mask], alpha=0.8, linewidth=2)
plt.grid(':', color='#80808080')
plt.legend(['Orig.', 'Reverb filtered'])
plt.show()